# RLHF Pipeline - Hybrid Approach

**Strategy**: Train on clean synthetic data, evaluate on real-world audio

## Workflow
1. **RLHF Training**: Collect feedback on synthetic base tones with text descriptions
2. **Evaluation**: Test improved model on REAL MusicCaps test set
3. **Analysis**: Does synthetic training → real improvement?

## Why This Works
- Clean RLHF signal (synthetic)
- Real evaluation (MusicCaps)
- Tests generalization\

In [1]:
# Cell 1: Imports
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from pathlib import Path
import json
from datetime import datetime
from tqdm import tqdm
import matplotlib.pyplot as plt
import IPython.display as ipd
from IPython.display import display
import librosa
import warnings
warnings.filterwarnings('ignore')

# Import model components
from lstmabar_model import LSTMABAR
from archetype_predictor import ArchetypePredictionHead, RLHFTrainer

print("✓ Imports loaded")
print(f"PyTorch version: {torch.__version__}")
print(f"Device: {'cuda' if torch.cuda.is_available() else 'cpu'}")

=== Testing Archetype Prediction ===
Predicted weights shape: torch.Size([8, 5])
Sample prediction: tensor([0.2076, 0.2413, 0.1859, 0.1924, 0.1729], grad_fn=<SelectBackward0>)
Sum of weights: 1.000000 (should be ~1.0)

Named predictions for sample 0:
  sine: 0.2259
  square: 0.3205
  sawtooth: 0.2257
  triangle: 0.1641
  noise: 0.0638

=== Testing Archetype Loss ===
MSE loss: 0.0191

=== Testing RLHF Trainer with Audio Playback ===

HUMAN FEEDBACK COLLECTION

Description: 'bright and cutting guitar tone'

Predicted Archetype Weights:
  sine      : ████ 0.208
  square    : ████ 0.241
  sawtooth  : ███ 0.186
  triangle  : ███ 0.192
  noise     : ███ 0.173

------------------------------------------------------------
AUDIO PLAYBACK
------------------------------------------------------------

▶️  ORIGINAL AUDIO:



▶️  TRANSFORMED AUDIO:



------------------------------------------------------------
RATING INSTRUCTIONS
------------------------------------------------------------
Rate how well the transformation matches the description:
  5 = Perfect match
  4 = Good match
  3 = Acceptable match
  2 = Poor match
  1 = Very poor match

✓ Feedback recorded: 5.0/5


=== Interactive RLHF Usage Example ===

# In Jupyter notebook, use this pattern:

# 1. Generate or load audio samples
original_audio = librosa.load('input.wav')[0]
transformed_audio = model.transform(original_audio, description)

# 2. Get embeddings and predictions
text_emb = text_encoder([description])
audio_emb = audio_encoder(torch.from_numpy(original_audio))
predicted_weights = predictor(text_emb, audio_emb)

# 3. Collect interactive feedback with audio playback
rating = rlhf_trainer.collect_feedback_with_audio(
description="bright and crunchy",
original_audio=original_audio,
transformed_audio=transformed_audio,
predicted_weights=predicted_weights[0].cpu().n

In [2]:
# Cell 2: Configuration

# Model config
MODEL_CONFIG = {
    'use_quantum_attention': True,
    'checkpoint_path': 'tuning_checkpoints/mids266final_bestmodel/best_model.pth',
    'sample_rate': 44100,
    'device': 'cuda' if torch.cuda.is_available() else 'cpu'
}

# Data paths
DATA_CONFIG = {
    'test_data_path': 'musiccaps_training_data_test.npz',
}

# RLHF config
RLHF_CONFIG = {
    'num_feedback_samples': 30,
    'reward_threshold': 3.0,
    'learning_rate': 1e-5,
    'num_rlhf_epochs': 5,
    'update_frequency': 5,
}

# Base tone config
BASE_TONE_CONFIG = {
    'duration': 2.0,
    'sample_rate': 44100,
    'fundamental_freq': 220.0,
}

# Output
OUTPUT_DIR = Path('rlhf_hybrid_results')
OUTPUT_DIR.mkdir(exist_ok=True)

print("✓ Configuration loaded")
print(f"  RLHF training: {RLHF_CONFIG['num_feedback_samples']} synthetic samples")
print(f"  Evaluation: Real test data from {DATA_CONFIG['test_data_path']}")
print(f"  Output dir: {OUTPUT_DIR}")

✓ Configuration loaded
  RLHF training: 30 synthetic samples
  Evaluation: Real test data from musiccaps_training_data_test.npz
  Output dir: rlhf_hybrid_results


In [3]:
# Cell 3: Load Model

print("="*60)
print("LOADING MODEL")
print("="*60)

device = torch.device(MODEL_CONFIG['device'])
print(f"\nUsing device: {device}")

# Load checkpoint
checkpoint_path = Path(MODEL_CONFIG['checkpoint_path'])
if not checkpoint_path.exists():
    raise FileNotFoundError(f"Checkpoint not found: {checkpoint_path}")

print(f"Loading from: {checkpoint_path}")
checkpoint = torch.load(checkpoint_path, map_location=device)

# Initialize model with Quantum attention
model = LSTMABAR(
    sample_rate=MODEL_CONFIG['sample_rate'],
    use_quantum_attention=MODEL_CONFIG['use_quantum_attention'],
    device=device
)

# Load weights
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()

print(f"✓ Model loaded successfully")
print(f"  Epoch: {checkpoint.get('epoch', 'unknown')}")
print(f"  Loss: {checkpoint.get('loss', 'unknown')}")
print(f"\n✓ Model has archetype predictor: {hasattr(model, 'archetype_predictor')}")
print(f"✓ Model has transform_audio: {hasattr(model, 'transform_audio')}")
print(f"✓ Model has inference: {hasattr(model, 'inference')}")

LOADING MODEL

Using device: cpu
Loading from: tuning_checkpoints/mids266final_bestmodel/best_model.pth
Loading text encoder: sentence-transformers/all-MiniLM-L6-v2
✓ Model loaded successfully
  Epoch: 7
  Loss: unknown

✓ Model has archetype predictor: True
✓ Model has transform_audio: True
✓ Model has inference: True


In [ ]:
# Cell 4: Load Real Test Data

print("="*60)
print("LOADING REAL TEST DATA")
print("="*60)

print(f"\nLoading from: {DATA_CONFIG['test_data_path']}")
test_data = np.load(DATA_CONFIG['test_data_path'], allow_pickle=True)

print(f"Available keys: {list(test_data.keys())}")

# Extract data
test_descriptions = test_data['descriptions']
test_archetypes = test_data['archetype_vectors']
test_audio_paths = test_data['audio_paths']
archetype_order = test_data['archetype_order']

print(f"\n✓ Metadata loaded: {len(test_descriptions)} test samples")
print(f"  Archetype order: {list(archetype_order)}")

# Load audio
def load_audio_from_paths(audio_paths, sample_rate=44100, max_duration=10.0):
    """Load audio from file paths"""
    loaded_audio = []
    failed = []
    max_length = int(sample_rate * max_duration)
    
    for audio_path in tqdm(audio_paths, desc="Loading test audio"):
        audio_path = str(audio_path)
        try:
            audio, sr = librosa.load(audio_path, sr=sample_rate, duration=max_duration)
            if len(audio) < max_length:
                audio = np.pad(audio, (0, max_length - len(audio)))
            else:
                audio = audio[:max_length]
            loaded_audio.append(audio)
        except Exception as e:
            if len(failed) < 5:
                print(f"\n  Failed: {audio_path}: {e}")
            loaded_audio.append(np.zeros(max_length))
            failed.append(audio_path)
    
    if failed:
        print(f"\n  {len(failed)}/{len(audio_paths)} files failed")
    
    return np.array(loaded_audio), failed

print("\n🎵 Loading test audio files...")
test_audio, test_failed = load_audio_from_paths(
    test_audio_paths, 
    MODEL_CONFIG['sample_rate']
)

print(f"\n✓ Test data loaded")
print(f"  Audio shape: {test_audio.shape}")
print(f"  Descriptions: {len(test_descriptions)}")
print(f"  Target archetypes: {test_archetypes.shape}")

LOADING REAL TEST DATA

Loading from: musiccaps_training_data_test.npz
Available keys: ['archetype_vectors', 'descriptions', 'audio_paths', 'archetype_order']

✓ Metadata loaded: 73 test samples
  Archetype order: [np.str_('sine'), np.str_('square'), np.str_('sawtooth'), np.str_('triangle'), np.str_('noise')]

🎵 Loading test audio files...


Loading test audio: 100%|██████████| 73/73 [00:02<00:00, 32.51it/s]



✓ Test data loaded
  Audio shape: (73, 441000)
  Descriptions: 73
  Target archetypes: (73, 5)


In [5]:
# Cell 5: Generate Synthetic Base Tones

def generate_base_tone(waveform_type='mixed', duration=2.0, sample_rate=44100, freq=220.0):
    """Generate synthetic base tone"""
    num_samples = int(duration * sample_rate)
    t = np.linspace(0, duration, num_samples, endpoint=False)
    
    if waveform_type == 'sine':
        audio = np.sin(2 * np.pi * freq * t)
    elif waveform_type == 'square':
        audio = np.sign(np.sin(2 * np.pi * freq * t))
    elif waveform_type == 'sawtooth':
        audio = 2 * (t * freq - np.floor(t * freq + 0.5))
    elif waveform_type == 'triangle':
        audio = 2 * np.abs(2 * (t * freq - np.floor(t * freq + 0.5))) - 1
    elif waveform_type == 'mixed':
        sine = np.sin(2 * np.pi * freq * t)
        square = np.sign(np.sin(2 * np.pi * freq * t))
        saw = 2 * (t * freq - np.floor(t * freq + 0.5))
        tri = 2 * np.abs(2 * (t * freq - np.floor(t * freq + 0.5))) - 1
        audio = (sine + square + saw + tri) / 4.0
    else:
        raise ValueError(f"Unknown waveform: {waveform_type}")
    
    # Normalize and fade
    audio = audio / (np.abs(audio).max() + 1e-8)
    fade_samples = int(0.01 * sample_rate)
    fade_in = np.linspace(0, 1, fade_samples)
    fade_out = np.linspace(1, 0, fade_samples)
    audio[:fade_samples] *= fade_in
    audio[-fade_samples:] *= fade_out
    
    return audio.astype(np.float32)

print("="*60)
print("GENERATING SYNTHETIC BASE TONES")
print("="*60)

default_base_tone = generate_base_tone(
    waveform_type='mixed',
    duration=BASE_TONE_CONFIG['duration'],
    sample_rate=BASE_TONE_CONFIG['sample_rate'],
    freq=BASE_TONE_CONFIG['fundamental_freq']
)

print(f"\n✓ Generated 'mixed' base tone: {default_base_tone.shape}")
print(f"  Used for all RLHF training transformations")

GENERATING SYNTHETIC BASE TONES

✓ Generated 'mixed' base tone: (88200,)
  Used for all RLHF training transformations


In [6]:
# Cell 6: Load Training Descriptions

TRAINING_DESCRIPTIONS = [
    "bright, cutting guitar tone",
    "warm, smooth guitar melody with gentle sustain",
    "harsh, digital synth-like guitar with buzzy edges",
    "sharp, metallic guitar plucks with quick decay",
    "crunchy overdriven guitar riffs with grit",
    "warm, mellow acoustic guitar strumming",
    "glassy, chiming harmonics on electric guitar",
    "thick, muffled palm-muted guitar rhythm",
    "raw, gritty blues guitar with expressive bends",
    "lush, chorus-soaked clean guitar chords",
    "dark, smoky cello ensemble in low register",
    "warm, smooth piano chords with soft transients",
    "bright, percussive piano stabs",
    "mellow, emotional grand piano melody",
    "dark, muted piano with felt-like tone",
    "sparkly, bell-like piano arpeggios",
    "soulful Hammond organ chords with slow rotary effect",
    "punchy, percussive clavinet riff with sharp attack",
    "dreamy, detuned synth-piano hybrid with soft transients",
    "crunchy, slightly distorted Rhodes with bite",
    "aggressive, bright synth lead with sharp harmonics",
    "warm, analog synth pad with gentle movement",
    "grainy, lofi synth lead with noise texture",
    "soft, airy synth lead with gentle brightness",
    "detuned, wobbling synth tone with drifting pitch",
    "raspy, resonant filter-sweep synth lead",
    "sparkly, crystalline synth plucks with short decay",
    "thick, wide supersaw lead with stereo spread",
    "hollow, formant-shifted synth tone with vowel-like quality",
    "lush, wide pad with dreamy texture",
    "dark, evolving ambient pad with low rumble",
    "celestial, shimmering pad with high-frequency sparkle",
    "hollow, airy pad with subtle modulation",
    "moody drone with slow-moving harmonics",
    "detuned, warm synth pad with analog drift",
    "ethereal, floating texture with soft overtones",
    "deep, warm sub-bass with smooth sine texture",
    "gritty, distorted bass with heavy saturation",
    "rubbery, bouncy synth bass with fast transients",
    "thick, resonant low-end bass with movement",
    "clean, round bass with gentle harmonics",
    "fuzzy, aggressive bass with buzz-saw texture",
    "white-noise burst with bright edges",
    "grainy, textured noise with soft filtering",
    "distorted, chaotic noise bed with harsh peaks",
    "warm, analog noise bed with subtle movement",
    "glitchy, stuttering texture with digital artifacts",
    "noisy, rough, chaotic saw-like texture",
    "tight, punchy kick drum with sharp attack",
    "snappy, bright snare with crisp transient",
    "warm, rounded tom hits with soft decay",
    "bright hi-hat pattern with metallic shimmer",
    "airy, crisp percussion groove with stereo shimmer",
    "tight, dry breakbeat with fast transients",
    "boomy, cinematic taiko-style drum hits",
    "gritty, overcompressed drum loop with pumping artifacts",
    "deep, resonant floor tom with long sustain",
    "sharp, percussive rimshot with strong transient",
    "bright, resonant violin melody",
    "soft, expressive flute line with airy tone",
    "warm clarinet phrase with smooth transitions",
    "dark, breathy saxophone melody",
    "bold brass stabs with powerful attack",
    "soft, lush orchestral strings with gentle swelling",
    "warm, woody double-bass plucks",
    "smooth horn section with warm resonance",
    "bright marimba strikes with clean attack",
    "glassy, shimmering celeste notes",
    "warm vibraphone chords with soft tremolo",
    "sharp, metallic bell hits with long decay",
    "hollow kalimba plucks with woody texture",
    "distant, echoing ambient chords",
    "soft, hazy reverb-washed tones",
    "crystalline ambient washes with airy diffusion",
    "deep, cavernous drone with subharmonics",
    "slow, swelling cinematic texture",
    "retro 80s synthwave lead with chorus",
    "dark industrial tone with metallic grit",
    "lofi, tape-warped acoustic texture",
    "robotic, vocoder-like synthetic tone",
    "electronic plucks with rapid transient snap",
    "warm, resonant plucks with rounded body",
    "metallic, atonal texture with shifting harmonics",
    "heavy, saturated resonant tone with compression pump"
]

# Randomly sample descriptions for this RLHF run
print("="*60)
print("TRAINING DESCRIPTIONS FOR RLHF")
print("="*60)
print(f"Total available: {len(TRAINING_DESCRIPTIONS)}")
print(f"Sampling: {RLHF_CONFIG['num_feedback_samples']} descriptions")
print()

# Set seed for reproducibility (change seed for different random samples)
import random
random_seed = 42  # Change this to get different samples each run
np.random.seed(random_seed)
random.seed(random_seed)

# Randomly sample from all descriptions
train_descriptions = random.sample(TRAINING_DESCRIPTIONS, RLHF_CONFIG['num_feedback_samples'])

print(f"✓ Randomly selected {len(train_descriptions)} descriptions")
print(f"  (To get different samples, change random_seed in this cell)")
print(f"\nFirst 5 selected:")
for i, desc in enumerate(train_descriptions[:5]):
    print(f"  {i+1}. {desc}")


TRAINING DESCRIPTIONS FOR RLHF
Total available: 84
Sampling: 30 descriptions

✓ Randomly selected 30 descriptions
  (To get different samples, change random_seed in this cell)

First 5 selected:
  1. warm, resonant plucks with rounded body
  2. dark, muted piano with felt-like tone
  3. sharp, metallic guitar plucks with quick decay
  4. ethereal, floating texture with soft overtones
  5. celestial, shimmering pad with high-frequency sparkle


---
## Helper Functions
---

In [ ]:
# Cell 7: Helper Functions

def transform_with_text(model, base_audio, description, device):
    """
    Transform base audio using text description
    
    CRITICAL FIX: Uses TEXT-ONLY archetype prediction!
    
    The model's archetype predictor was trained with BOTH text and audio embeddings.
    When using the same base tone for all descriptions, the audio embedding is
    constant, causing identical predictions. We bypass this by using ZERO audio
    embeddings, forcing the model to rely only on text.
    """
    # Convert audio to tensor
    audio_tensor = torch.from_numpy(base_audio).unsqueeze(0).float().to(device)
    
    with torch.no_grad():
        # Get text embedding
        text_emb = model.encode_text([description])
        
        # CRITICAL FIX: Use ZERO audio embedding to force text-only prediction
        # This prevents the constant base tone from dominating predictions
        zero_audio_emb = torch.zeros_like(text_emb).to(device)
        
        # Predict archetypes using text-only (with zero audio embedding)
        archetype_weights = model.predict_archetypes(text_emb, zero_audio_emb)
        
        # Transform audio using DDSP
        transformed = model.transform_audio(audio_tensor, archetype_weights)
    
    # Convert back
    transformed_audio = transformed.squeeze(0).cpu().numpy()
    archetype_weights_np = archetype_weights.squeeze(0).cpu().numpy()
    
    return transformed_audio, archetype_weights_np


def evaluate_on_real_test_set(model, test_audio, test_descriptions, test_archetypes, device):
    """Evaluate model on real test data"""
    model.eval()
    results = []
    
    with torch.no_grad():
        for i in tqdm(range(len(test_audio)), desc="Evaluating"):
            audio = test_audio[i]
            description = test_descriptions[i]
            target_archetypes = test_archetypes[i]
            
            # Model prediction - use REAL audio embeddings for evaluation
            audio_tensor = torch.from_numpy(audio).unsqueeze(0).float().to(device)
            _, metadata = model.inference([description], audio_tensor)
            pred_arch = metadata['predicted_weights'][0]
            
            # Metrics
            arch_mse = np.mean((pred_arch - target_archetypes) ** 2)
            cos_sim = np.dot(pred_arch, target_archetypes) / (
                np.linalg.norm(pred_arch) * np.linalg.norm(target_archetypes) + 1e-8
            )
            pred_max = np.argmax(pred_arch)
            target_max = np.argmax(target_archetypes)
            arch_correct = int(pred_max == target_max)
            
            results.append({
                'description': description,
                'predicted_archetypes': pred_arch.tolist(),
                'target_archetypes': target_archetypes.tolist(),
                'archetype_mse': float(arch_mse),
                'cosine_similarity': float(cos_sim),
                'archetype_correct': arch_correct
            })
    
    # Aggregate
    avg_mse = np.mean([r['archetype_mse'] for r in results])
    avg_cos_sim = np.mean([r['cosine_similarity'] for r in results])
    arch_accuracy = np.mean([r['archetype_correct'] for r in results]) * 100
    
    return results, {
        'avg_mse': avg_mse,
        'avg_cosine_similarity': avg_cos_sim,
        'archetype_accuracy': arch_accuracy
    }


def play_comparison(original, transformed, sample_rate=44100):
    """Play audio comparison"""
    print("🎵 ORIGINAL:")
    display(ipd.Audio(original, rate=sample_rate))
    print("\n🎵 TRANSFORMED:")
    display(ipd.Audio(transformed, rate=sample_rate))


def get_rating_from_user(description, attempt=0, max_attempts=3):
    """Get rating from user"""
    print("\n" + "="*60)
    print("RATE THE TRANSFORMATION")
    print("="*60)
    print(f"Description: '{description}'")
    print("\nHow well does transformation match description?")
    print("  5 = Perfect")
    print("  4 = Good")
    print("  3 = Acceptable")
    print("  2 = Poor")
    print("  1 = Very poor")
    
    try:
        rating = input("\nYour rating (1-5): ").strip()
        rating = int(rating)
        if rating not in [1, 2, 3, 4, 5]:
            print("  Rating must be 1-5")
            if attempt < max_attempts:
                return get_rating_from_user(description, attempt + 1, max_attempts)
            return 3
        return rating
    except ValueError:
        print("  Please enter 1-5")
        if attempt < max_attempts:
            return get_rating_from_user(description, attempt + 1, max_attempts)
        return 3
    except KeyboardInterrupt:
        print("\n\n  Interrupted")
        raise


def compute_reward(rating, threshold=3.0):
    """Convert rating to reward"""
    return (rating - 3.0) / 2.0


print("✓ Helper functions loaded")
print("✓ Using TEXT-ONLY archetype prediction for RLHF!")

✓ Helper functions loaded
✓ Using TEXT-ONLY archetype prediction for RLHF!


---
## Pre-RLHF Baseline
---

In [8]:
# Cell 8: Pre-RLHF Baseline

print("="*60)
print("PRE-RLHF BASELINE EVALUATION")
print("="*60)
print(f"\nEvaluating on {len(test_audio)} REAL test samples")
print("Baseline before RLHF...\n")

baseline_results, baseline_metrics = evaluate_on_real_test_set(
    model, test_audio, test_descriptions, test_archetypes, device
)

print("\n" + "="*60)
print("BASELINE RESULTS")
print("="*60)
print(f"MSE:              {baseline_metrics['avg_mse']:.6f}")
print(f"Cosine sim:       {baseline_metrics['avg_cosine_similarity']:.4f}")
print(f"Accuracy:         {baseline_metrics['archetype_accuracy']:.2f}%")

with open(OUTPUT_DIR / 'baseline_results.json', 'w') as f:
    json.dump({
        'metrics': baseline_metrics,
        'detailed_results': baseline_results[:10]
    }, f, indent=2)

print(f"\n✓ Saved: {OUTPUT_DIR / 'baseline_results.json'}")

PRE-RLHF BASELINE EVALUATION

Evaluating on 73 REAL test samples
Baseline before RLHF...



Evaluating: 100%|██████████| 73/73 [00:51<00:00,  1.43it/s]


BASELINE RESULTS
MSE:              0.105025
Cosine sim:       0.5820
Accuracy:         31.51%

✓ Saved: rlhf_hybrid_results/baseline_results.json


---
## RLHF Training (Synthetic)
---

In [14]:
# Cell 9: RLHF Training

print("="*60)
print("RLHF FEEDBACK COLLECTION")
print("="*60)
print(f"\nRating {len(train_descriptions)} transformations")
print("Training on SYNTHETIC, testing on REAL!\n")
print(f"Time: ~{len(train_descriptions) * 1.5:.0f}-{len(train_descriptions) * 2:.0f} min\n")

input("Press Enter to start...")

# Setup
feedback_history = []
optimizer = optim.Adam(model.parameters(), lr=RLHF_CONFIG['learning_rate'])
archetype_names = ['sine', 'square', 'sawtooth', 'triangle', 'noise']

for sample_idx, description in enumerate(train_descriptions):
    print(f"\n{'='*60}")
    print(f"Sample {sample_idx + 1}/{len(train_descriptions)}")
    print(f"{'='*60}")
    
    # Keep model in EVAL mode for transformation (BatchNorm needs batch_size > 1 in train mode)
    model.eval()
    
    # Transform
    transformed, archetypes = transform_with_text(
        model, default_base_tone, description, device
    )
    
    print(f"\nDescription: '{description}'")
    print(f"\nPredicted archetypes:")
    for name, weight in zip(archetype_names, archetypes):
        bar = '█' * int(weight * 30)
        print(f"  {name:10s}: {weight:.3f} {bar}")
    
    # Detailed audio diagnostics
    print(f"\n🔍 Transformation diagnostics:")
    orig_mean = np.mean(default_base_tone)
    orig_std = np.std(default_base_tone)
    orig_max = np.max(np.abs(default_base_tone))
    trans_mean = np.mean(transformed)
    trans_std = np.std(transformed)
    trans_max = np.max(np.abs(transformed))
    
    print(f"  Original:    mean={orig_mean:.6f}, std={orig_std:.6f}, max={orig_max:.6f}")
    print(f"  Transformed: mean={trans_mean:.6f}, std={trans_std:.6f}, max={trans_max:.6f}")
    
    # Check if actually different
    min_len = min(len(default_base_tone), len(transformed))
    if np.allclose(default_base_tone[:min_len], transformed[:min_len], atol=1e-4):
        print(f"  ⚠️  WARNING: Audio appears UNCHANGED!")
        print(f"  This means DDSP didn't transform it.")
    else:
        # Calculate difference
        diff = np.abs(default_base_tone[:min_len] - transformed[:min_len])
        print(f"  ✓ Audio IS different (avg diff: {np.mean(diff):.6f})")
        print(f"  Transformation strength: {(np.mean(diff) / orig_std * 100):.1f}% of original std")
    
    # Play
    print("\n" + "-"*60)
    play_comparison(default_base_tone, transformed, MODEL_CONFIG['sample_rate'])
    print("-"*60)
    
    # Rate
    rating = get_rating_from_user(description)
    reward = compute_reward(rating, RLHF_CONFIG['reward_threshold'])
    
    print(f"\n✓ Rating: {rating}/5, Reward: {reward:+.2f}")
    
    # Store
    feedback_history.append({
        'sample_idx': sample_idx,
        'description': description,
        'archetypes': archetypes.tolist(),
        'rating': int(rating),
        'reward': float(reward),
        'timestamp': datetime.now().isoformat()
    })
    
    # Update model
    if (sample_idx + 1) % RLHF_CONFIG['update_frequency'] == 0:
        print(f"\n🔄 Updating model...")
        
        # Switch to TRAIN mode for gradient updates
        model.train()
        
        recent = feedback_history[-RLHF_CONFIG['update_frequency']:]
        avg_reward = np.mean([f['reward'] for f in recent])
        print(f"  Recent avg reward: {avg_reward:+.3f}")
        
        # Policy gradient
        for epoch in range(RLHF_CONFIG['num_rlhf_epochs']):
            epoch_loss = 0.0
            
            # Create batch from recent feedback for BatchNorm
            batch_texts = [f['description'] for f in recent]
            batch_rewards = torch.tensor([f['reward'] for f in recent]).float().to(device)
            
            # Get text embeddings for batch
            text_emb_batch = model.encode_text(batch_texts)
            
            # CRITICAL FIX: Use ZERO audio embeddings (TEXT-ONLY prediction)
            zero_audio_emb_batch = torch.zeros_like(text_emb_batch).to(device)
            
            # Predict archetypes using text-only for entire batch
            predicted_archetypes_batch = model.predict_archetypes(text_emb_batch, zero_audio_emb_batch)
            
            # Policy gradient loss for batch
            log_probs = torch.log(predicted_archetypes_batch + 1e-8).sum(dim=1)
            loss = -(log_probs * batch_rewards).mean()
            
            optimizer.zero_grad()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            
            epoch_loss += loss.item()
        
        print(f"  ✓ Updated (loss: {epoch_loss/RLHF_CONFIG['num_rlhf_epochs']:.4f})")
        
        # Switch back to EVAL mode
        model.eval()

print(f"\n\n{'='*60}")
print("RLHF COMPLETE")
print(f"{'='*60}")
print(f"Total: {len(feedback_history)}")
print(f"Avg rating: {np.mean([f['rating'] for f in feedback_history]):.2f}")
print(f"Avg reward: {np.mean([f['reward'] for f in feedback_history]):+.3f}")
print(f"Positive (≥3): {sum(1 for f in feedback_history if f['rating'] >= 3)}/{len(feedback_history)}")

with open(OUTPUT_DIR / 'feedback_history.json', 'w') as f:
    json.dump(feedback_history, f, indent=2)
print(f"\n✓ Saved: {OUTPUT_DIR / 'feedback_history.json'}")

RLHF FEEDBACK COLLECTION

Rating 30 transformations
Training on SYNTHETIC, testing on REAL!

Time: ~45-60 min


Sample 1/30

Description: 'warm, resonant plucks with rounded body'

Predicted archetypes:
  sine      : 0.206 ██████
  square    : 0.121 ███
  sawtooth  : 0.245 ███████
  triangle  : 0.225 ██████
  noise     : 0.204 ██████

🔍 Transformation diagnostics:
  Original:    mean=0.000112, std=0.746236, max=1.000000
  Transformed: mean=-0.003570, std=0.634998, max=0.950000
  ✓ Audio IS different (avg diff: 0.104485)
  Transformation strength: 14.0% of original std

------------------------------------------------------------
🎵 ORIGINAL:



🎵 TRANSFORMED:


------------------------------------------------------------

RATE THE TRANSFORMATION
Description: 'warm, resonant plucks with rounded body'

How well does transformation match description?
  5 = Perfect
  4 = Good
  3 = Acceptable
  2 = Poor
  1 = Very poor
⚠️  Please enter 1-5

RATE THE TRANSFORMATION
Description: 'warm, resonant plucks with rounded body'

How well does transformation match description?
  5 = Perfect
  4 = Good
  3 = Acceptable
  2 = Poor
  1 = Very poor

✓ Rating: 1/5, Reward: -1.00

Sample 2/30

Description: 'dark, muted piano with felt-like tone'

Predicted archetypes:
  sine      : 0.220 ██████
  square    : 0.155 ████
  sawtooth  : 0.221 ██████
  triangle  : 0.269 ████████
  noise     : 0.134 ████

🔍 Transformation diagnostics:
  Original:    mean=0.000112, std=0.746236, max=1.000000
  Transformed: mean=-0.004499, std=0.661856, max=0.950000
  ✓ Audio IS different (avg diff: 0.078388)
  Transformation strength: 10.5% of original std

-----------------------------


🎵 TRANSFORMED:


------------------------------------------------------------

RATE THE TRANSFORMATION
Description: 'dark, muted piano with felt-like tone'

How well does transformation match description?
  5 = Perfect
  4 = Good
  3 = Acceptable
  2 = Poor
  1 = Very poor
⚠️  Please enter 1-5

RATE THE TRANSFORMATION
Description: 'dark, muted piano with felt-like tone'

How well does transformation match description?
  5 = Perfect
  4 = Good
  3 = Acceptable
  2 = Poor
  1 = Very poor

✓ Rating: 1/5, Reward: -1.00

Sample 3/30

Description: 'sharp, metallic guitar plucks with quick decay'

Predicted archetypes:
  sine      : 0.254 ███████
  square    : 0.144 ████
  sawtooth  : 0.232 ██████
  triangle  : 0.216 ██████
  noise     : 0.154 ████

🔍 Transformation diagnostics:
  Original:    mean=0.000112, std=0.746236, max=1.000000
  Transformed: mean=-0.004112, std=0.649807, max=0.950000
  ✓ Audio IS different (avg diff: 0.089820)
  Transformation strength: 12.0% of original std

-------------------------

...


🎵 TRANSFORMED:


------------------------------------------------------------

RATE THE TRANSFORMATION
Description: 'hollow, formant-shifted synth tone with vowel-like quality'

How well does transformation match description?
  5 = Perfect
  4 = Good
  3 = Acceptable
  2 = Poor
  1 = Very poor

✓ Rating: 1/5, Reward: -1.00

Sample 7/30

Description: 'punchy, percussive clavinet riff with sharp attack'

Predicted archetypes:
  sine      : 0.210 ██████
  square    : 0.178 █████
  sawtooth  : 0.271 ████████
  triangle  : 0.203 ██████
  noise     : 0.139 ████

🔍 Transformation diagnostics:
  Original:    mean=0.000112, std=0.746236, max=1.000000
  Transformed: mean=-0.005073, std=0.665432, max=0.950000
  ✓ Audio IS different (avg diff: 0.074711)
  Transformation strength: 10.0% of original std

------------------------------------------------------------
🎵 ORIGINAL:



🎵 TRANSFORMED:


------------------------------------------------------------

RATE THE TRANSFORMATION
Description: 'punchy, percussive clavinet riff with sharp attack'

How well does transformation match description?
  5 = Perfect
  4 = Good
  3 = Acceptable
  2 = Poor
  1 = Very poor


⚠️  Interrupted


KeyboardInterrupt: Interrupted by user

---
## Post-RLHF Evaluation (Real)
---

In [ ]:
# Cell 10: Post-RLHF Evaluation

print("="*60)
print("POST-RLHF EVALUATION")
print("="*60)
print(f"\nEvaluating on {len(test_audio)} REAL test samples")
print("Did synthetic training help?\n")

postrlhf_results, postrlhf_metrics = evaluate_on_real_test_set(
    model, test_audio, test_descriptions, test_archetypes, device
)

print("\n" + "="*60)
print("POST-RLHF RESULTS")
print("="*60)
print(f"MSE:              {postrlhf_metrics['avg_mse']:.6f}")
print(f"Cosine sim:       {postrlhf_metrics['avg_cosine_similarity']:.4f}")
print(f"Accuracy:         {postrlhf_metrics['archetype_accuracy']:.2f}%")

with open(OUTPUT_DIR / 'postrlhf_results.json', 'w') as f:
    json.dump({
        'metrics': postrlhf_metrics,
        'detailed_results': postrlhf_results[:10]
    }, f, indent=2)

print(f"\n✓ Saved: {OUTPUT_DIR / 'postrlhf_results.json'}")

In [ ]:
# Cell 11: Compare Results

print("="*60)
print("PRE vs POST RLHF COMPARISON")
print("="*60)
print("\nTrained: Synthetic (30 samples)")
print("Tested:  Real MusicCaps (73 samples)\n")

print("Metric                  Before         After          Change")
print("-" * 65)

mse_before = baseline_metrics['avg_mse']
mse_after = postrlhf_metrics['avg_mse']
mse_change = mse_after - mse_before
mse_pct = (mse_change / mse_before) * 100 if mse_before > 0 else 0
print(f"MSE:                    {mse_before:.6f}     {mse_after:.6f}     {mse_change:+.6f} ({mse_pct:+.1f}%)")

cos_before = baseline_metrics['avg_cosine_similarity']
cos_after = postrlhf_metrics['avg_cosine_similarity']
cos_change = cos_after - cos_before
cos_pct = (cos_change / cos_before) * 100 if cos_before > 0 else 0
print(f"Cosine Sim:             {cos_before:.6f}     {cos_after:.6f}     {cos_change:+.6f} ({cos_pct:+.1f}%)")

acc_before = baseline_metrics['archetype_accuracy']
acc_after = postrlhf_metrics['archetype_accuracy']
acc_change = acc_after - acc_before
print(f"Accuracy:               {acc_before:.2f}%        {acc_after:.2f}%        {acc_change:+.2f}%")

print("\n" + "="*60)
print("INTERPRETATION")
print("="*60)

if mse_change < 0:
    print("✓ MSE decreased (better predictions)")
else:
    print("⚠ MSE increased")

if cos_change > 0:
    print("✓ Cosine similarity increased (better alignment)")
else:
    print("⚠ Cosine similarity decreased")

if acc_change > 0:
    print("✓ Accuracy increased")
else:
    print("⚠ Accuracy decreased")

print("\nSynthetic → Real transfer: ", end="")
if mse_change < 0 and cos_change > 0:
    print("✓ SUCCESS!")
elif abs(mse_change/mse_before) < 0.05:
    print("✓ STABLE (didn't overfit)")
else:
    print("⚠ MIXED RESULTS")

# Save comparison
comparison = {
    'training': {
        'data_type': 'synthetic',
        'num_samples': len(feedback_history),
        'avg_rating': float(np.mean([f['rating'] for f in feedback_history])),
        'positive_pct': float(100 * sum(1 for f in feedback_history if f['rating'] >= 3) / len(feedback_history))
    },
    'evaluation': {
        'data_type': 'real',
        'num_samples': len(test_audio)
    },
    'metrics': {
        'before': baseline_metrics,
        'after': postrlhf_metrics,
        'changes': {
            'mse_change': float(mse_change),
            'mse_pct': float(mse_pct),
            'cos_change': float(cos_change),
            'cos_pct': float(cos_pct),
            'acc_change': float(acc_change)
        }
    }
}

with open(OUTPUT_DIR / 'comparison.json', 'w') as f:
    json.dump(comparison, f, indent=2)

print(f"\n✓ Saved: {OUTPUT_DIR / 'comparison.json'}")

---
## Save & Visualize
---

In [ ]:
# Cell 12: Save Model

print("="*60)
print("SAVING MODEL")
print("="*60)

final_model_path = OUTPUT_DIR / 'rlhf_final_model.pth'
torch.save({
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'config': MODEL_CONFIG,
    'rlhf_config': RLHF_CONFIG,
    'training_summary': comparison['training'],
    'evaluation_summary': comparison['evaluation'],
    'timestamp': datetime.now().isoformat()
}, final_model_path)

print(f"✓ Saved: {final_model_path}")
print(f"  Size: {final_model_path.stat().st_size / 1024 / 1024:.1f} MB")

In [ ]:
# Cell 13: Visualizations

print("="*60)
print("VISUALIZATIONS")
print("="*60)

fig, axes = plt.subplots(2, 3, figsize=(18, 10))
fig.suptitle('Hybrid RLHF: Synthetic Training → Real Evaluation', fontsize=16, fontweight='bold')

# Training plots
ratings = [f['rating'] for f in feedback_history]
rewards = [f['reward'] for f in feedback_history]

axes[0, 0].hist(ratings, bins=5, range=(0.5, 5.5), edgecolor='black', alpha=0.7, color='skyblue')
axes[0, 0].set_title('Training: Ratings (Synthetic)')
axes[0, 0].set_xlabel('Rating')
axes[0, 0].set_xticks([1, 2, 3, 4, 5])
axes[0, 0].grid(axis='y', alpha=0.3)

axes[0, 1].plot(rewards, 'o-', alpha=0.6, color='green')
axes[0, 1].axhline(y=0, color='r', linestyle='--', alpha=0.5)
axes[0, 1].set_title('Training: Rewards')
axes[0, 1].set_xlabel('Sample')
axes[0, 1].grid(alpha=0.3)

cumulative = np.cumsum([1 if r >= 3 else 0 for r in ratings])
axes[0, 2].plot(cumulative, 'o-', color='green', alpha=0.6)
axes[0, 2].set_title('Training: Cumulative Positive')
axes[0, 2].set_xlabel('Sample')
axes[0, 2].grid(alpha=0.3)

# Evaluation plots
axes[1, 0].bar(['Before', 'After'], [mse_before, mse_after], color=['coral', 'lightgreen'], edgecolor='black')
axes[1, 0].set_title('Eval: MSE (Real Test)')
axes[1, 0].grid(axis='y', alpha=0.3)

axes[1, 1].bar(['Before', 'After'], [cos_before, cos_after], color=['coral', 'lightgreen'], edgecolor='black')
axes[1, 1].set_title('Eval: Cosine Sim (Real Test)')
axes[1, 1].grid(axis='y', alpha=0.3)

axes[1, 2].bar(['Before', 'After'], [acc_before, acc_after], color=['coral', 'lightgreen'], edgecolor='black')
axes[1, 2].set_title('Eval: Accuracy (Real Test)')
axes[1, 2].grid(axis='y', alpha=0.3)

plt.tight_layout()
viz_path = OUTPUT_DIR / 'rlhf_hybrid_analysis.png'
plt.savefig(viz_path, dpi=150, bbox_inches='tight')
print(f"\n✓ Saved: {viz_path}")
plt.show()

In [ ]:
# Cell 14: Final Report

report = f"""
HYBRID RLHF - FINAL REPORT
{'='*70}

APPROACH
--------
Training:   Synthetic base tones + text descriptions ({len(feedback_history)} samples)
Evaluation: Real MusicCaps test audio ({len(test_audio)} samples)
Goal:       Test if synthetic RLHF → real improvement

TRAINING (Synthetic)
--------------------
Samples:    {len(feedback_history)}
Avg rating: {np.mean([f['rating'] for f in feedback_history]):.2f}/5
Positive:   {sum(1 for f in feedback_history if f['rating'] >= 3)}/{len(feedback_history)} ({100*sum(1 for f in feedback_history if f['rating'] >= 3)/len(feedback_history):.1f}%)

EVALUATION (Real Test Data)
---------------------------
Metric              Before         After          Change
──────────────────────────────────────────────────────────────
MSE:                {mse_before:.6f}     {mse_after:.6f}     {mse_change:+.6f} ({mse_pct:+.1f}%)
Cosine Sim:         {cos_before:.6f}     {cos_after:.6f}     {cos_change:+.6f} ({cos_pct:+.1f}%)
Accuracy:           {acc_before:.2f}%        {acc_after:.2f}%        {acc_change:+.2f}%

CONCLUSION
----------
"""

if mse_change < 0 and cos_change > 0:
    report += "✓ SUCCESS: Synthetic training improved real performance!\n"
elif abs(mse_change / mse_before) < 0.05:
    report += "✓ STABLE: Model maintained performance (didn't overfit)\n"
else:
    report += "⚠ MIXED: Some metrics improved, others declined\n"

report += f"""
This demonstrates {'successful' if mse_change < 0 else 'the challenge of'} transfer learning
from synthetic to real audio in RLHF settings.

FILES
-----
Model:       {OUTPUT_DIR / 'rlhf_final_model.pth'}
Feedback:    {OUTPUT_DIR / 'feedback_history.json'}
Baseline:    {OUTPUT_DIR / 'baseline_results.json'}
Post-RLHF:   {OUTPUT_DIR / 'postrlhf_results.json'}
Comparison:  {OUTPUT_DIR / 'comparison.json'}
Viz:         {OUTPUT_DIR / 'rlhf_hybrid_analysis.png'}

{datetime.now().strftime('%Y-%m-%d %H:%M:%S')}
"""

print(report)

with open(OUTPUT_DIR / 'final_report.txt', 'w') as f:
    f.write(report)

print(f"✓ Saved: {OUTPUT_DIR / 'final_report.txt'}")
print("\n" + "="*60)
print("✅ COMPLETE!")
print("="*60)
print(f"\nAll outputs: {OUTPUT_DIR}/")
print("\n🎉 Hybrid RLHF done! Trained synthetic, tested real.")